**Magic Commands that are useful**
* autoreload is helpful because it reloads packages automatically once they are changed (i.e., pyrankability)
* matploblib inline is helpful because it tells the notebook to output figures to the notebook

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

**Convention on imports**
* It is convention to put builtin imports first
* Then standard third party imports
* Then custom imports (pyrankability)

In [2]:
import copy
import os

In [3]:
import pandas as pd
import numpy as np

In [4]:
import sys
# We need to include the path to pyrankability. This could be different for Tim, but altneratively he could point to your copy
sys.path.insert(0,"/disk/home/amy/rankability_toolbox_dev")

In [5]:
sys.path.insert(0,"/disk/home/amy/sensitivity_study/src")

In [6]:
from sensitivity_tests import *

In [7]:
import pyrankability

In [8]:
from base import *

In [9]:
games={}
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]
for year in years:
    games[year] = read_data('data/%steams.txt'%year,'data/%sgames.txt'%year,'data/%sMadnessTeams.txt'%year)
games[year]

,team2,team1,notsure1,date,H_A_N1,points1,H_A_N2,points2,team1_name,team2_name,team1_madness,team2_madness
0,1,16,737011,2017-11-13,1,83,-1,69,Arkansas_St,Abilene_Chr,0,0
1,1,41,737114,2018-02-24,-1,74,1,72,Cent_Arkansas,Abilene_Chr,0,0
3,1,143,737018,2017-11-20,-1,75,1,67,Lipscomb,Abilene_Chr,1,0
4,1,143,737045,2017-12-17,1,67,-1,65,Lipscomb,Abilene_Chr,1,0
5,1,199,737056,2017-12-28,1,77,-1,74,New_Orleans,Abilene_Chr,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5535,351,318,737048,2017-12-20,1,91,-1,74,Utah_St,Youngstown_St,0,0
5536,351,339,737086,2018-01-27,1,85,-1,67,WI_Green_Bay,Youngstown_St,0,0
5537,351,340,737084,2018-01-25,1,66,-1,55,WI_Milwaukee,Youngstown_St,0,0
5538,351,347,737074,2018-01-15,1,77,-1,67,Wright_St,Youngstown_St,1,0


In [97]:
for_example = games[year][["team1_name","team2_name","points1","points2"]]
for_example.columns = ["Team 1","Team 2","Team 1 Score","Team 2 Score"]
for_display=for_example.sample(frac=0.9).head()
display(for_display)
print(for_display.to_latex())

,Team 1,Team 2,Team 1 Score,Team 2 Score
2100,E_Illinois,IUPUI,80,79
199,UCLA,Arizona,82,74
5036,Coastal_Car,UT_San_Antonio,83,69
1260,James_Madison,Drexel,75,73
1158,Howard,Delaware_St,93,74


\begin{tabular}{lllrr}
\toprule
{} &         Team 1 &          Team 2 &  Team 1 Score &  Team 2 Score \\
\midrule
2100 &     E\_Illinois &           IUPUI &            80 &            79 \\
199  &           UCLA &         Arizona &            82 &            74 \\
5036 &    Coastal\_Car &  UT\_San\_Antonio &            83 &            69 \\
1260 &  James\_Madison &          Drexel &            75 &            73 \\
1158 &         Howard &     Delaware\_St &            93 &            74 \\
\bottomrule
\end{tabular}



In [98]:
year

'2018'

In [10]:
for year in games.keys():
    print("Dates for %s:"%year,games[year]['date'].min(),games[year]['date'].max())

Dates for 2002: 2001-11-08 00:00:00 2002-03-03 00:00:00
Dates for 2003: 2002-11-14 00:00:00 2003-03-09 00:00:00
Dates for 2004: 2003-11-13 00:00:00 2004-03-07 00:00:00
Dates for 2005: 2004-11-11 00:00:00 2005-03-06 00:00:00
Dates for 2006: 2005-11-08 00:00:00 2006-03-05 00:00:00
Dates for 2007: 2006-11-07 00:00:00 2007-03-04 00:00:00
Dates for 2008: 2007-11-05 00:00:00 2008-03-09 00:00:00
Dates for 2009: 2008-11-10 00:00:00 2009-03-08 00:00:00
Dates for 2010: 2009-11-09 00:00:00 2010-03-07 00:00:00
Dates for 2011: 2010-11-08 00:00:00 2011-03-06 00:00:00
Dates for 2012: 2011-11-07 00:00:00 2012-03-04 00:00:00
Dates for 2013: 2012-11-09 00:00:00 2013-03-10 00:00:00
Dates for 2014: 2013-11-08 00:00:00 2014-03-09 00:00:00
Dates for 2015: 2014-11-14 00:00:00 2015-03-08 00:00:00
Dates for 2016: 2015-11-13 00:00:00 2016-03-06 00:00:00
Dates for 2017: 2016-11-11 00:00:00 2017-03-05 00:00:00
Dates for 2018: 2017-11-10 00:00:00 2018-03-04 00:00:00


In [11]:

def support_map_vectorized1(linked,num_indirect_equal_direct=3):
    # columns
    # 'team_j', 'team_i_name', 'team_i_score', 'team_i_H_A_N',
    # 'team_j_i_score', 'team_j_i_H_A_N', 'game_i_j', 'team_k_name',
    # 'team_k_score', 'team_k_H_A_N', 'team_j_k_score', 'team_j_k_H_A_N',
    # 'game_k_j'
    linked["direct"] = linked["team_i_name"] == linked["team_k_name"]
    # | (linked["team_i_name"] == linked["team_j_k_name"]) | (linked["team_k_name"] == linked["team_j_k_name"])
    for_index1 = linked[["team_i_name","team_k_name"]].copy()
    for_index1.loc[linked["direct"]] = linked.loc[linked["direct"],["team_i_name","team_j_name"]]
    for_index1.columns = ["team1","team2"]
    for_index2 = linked[["team_k_name","team_i_name"]].copy()
    for_index2.loc[linked["direct"]] = linked.loc[linked["direct"],["team_j_name","team_i_name"]]
    for_index2.columns = ["team1","team2"]
    index_ik = pd.MultiIndex.from_frame(for_index1,sortorder=0)
    index_ki = pd.MultiIndex.from_frame(for_index2,sortorder=0)
    
    #######################################
    # part to modify
    # direct
    d_ik = linked['team_i_score'] - linked['team_j_i_score']
    direct_thres = 1
    support_ik = num_indirect_equal_direct*(linked["direct"] & (d_ik > direct_thres)).astype(int)
    support_ki = num_indirect_equal_direct*(linked["direct"] & (d_ik < -direct_thres)).astype(int)

    # indirect
    d_ij = linked["team_i_score"] - linked["team_j_i_score"]
    d_kj = linked["team_k_score"] - linked["team_j_k_score"]
    
    # always a positive and it captures that if i beat j by 5 points and k beat j by 2 points then this spread is 3
    spread = np.abs(d_ij - d_kj) 
    
    support_ik += ((~linked["direct"]) & (d_ij > 0) & (d_kj > 0) & (d_ij > d_kj) & (spread > 10)).astype(int)
    support_ik += ((~linked["direct"]) & (d_ij < 0) & (d_kj < 0) & (d_ij > d_kj) & (spread > 15)).astype(int)
    support_ik += ((~linked["direct"]) & (d_ij > 0) & (d_kj < 0) & (spread > 2)).astype(int)
    
    support_ki += ((~linked["direct"]) & (d_kj > 0) & (d_ij > 0) & (d_kj > d_ij) & (spread > 10)).astype(int)
    support_ki += ((~linked["direct"]) & (d_kj < 0) & (d_ij < 0) & (d_kj > d_ij) & (spread > 15)).astype(int)
    support_ki += ((~linked["direct"]) & (d_kj > 0) & (d_ij < 0) & (spread > 2)).astype(int)
    
    # end part to modify
    #######################################    
    linked["support_ik"]=support_ik
    linked["index_ik"]=index_ik
    linked["support_ki"]=support_ki
    linked["index_ki"]=index_ki
        
    #prepare_ret = linked.drop_duplicates(subset='games', keep='first')[["index_ik","index_ki","support_ik","support_ki"]]
    ret1 = linked.set_index(index_ik)["support_ik"]
    ret2 = linked.set_index(index_ki)["support_ki"]
    ret = ret1.append(ret2)
    ret = ret.groupby(level=[0,1]).sum()

    return ret



In [118]:

def support_map_vectorized_direct(linked,direct_thres=1):
    # columns
    # 'team_j', 'team_i_name', 'team_i_score', 'team_i_H_A_N',
    # 'team_j_i_score', 'team_j_i_H_A_N', 'game_i_j', 'team_k_name',
    # 'team_k_score', 'team_k_H_A_N', 'team_j_k_score', 'team_j_k_H_A_N',
    # 'game_k_j'
    linked["direct"] = linked["team_i_name"] == linked["team_k_name"]
    linked = linked.loc[linked["direct"]].copy()
    # | (linked["team_i_name"] == linked["team_j_k_name"]) | (linked["team_k_name"] == linked["team_j_k_name"])
    for_index1 = linked[["team_i_name","team_k_name"]].copy()
    for_index1.loc[linked["direct"]] = linked.loc[linked["direct"],["team_i_name","team_j_name"]]
    for_index1.columns = ["team1","team2"]
    for_index2 = linked[["team_k_name","team_i_name"]].copy()
    for_index2.loc[linked["direct"]] = linked.loc[linked["direct"],["team_j_name","team_i_name"]]
    for_index2.columns = ["team1","team2"]
    index_ik = pd.MultiIndex.from_frame(for_index1,sortorder=0)
    index_ki = pd.MultiIndex.from_frame(for_index2,sortorder=0)
    
    #######################################
    # part to modify
    # direct
    d_ik = linked['team_i_score'] - linked['team_j_i_score']
    support_ik = (linked["direct"] & (d_ik > direct_thres)).astype(int)
    support_ki = (linked["direct"] & (d_ik < -direct_thres)).astype(int)
    
    # end part to modify
    #######################################    
    linked["support_ik"]=support_ik
    linked["index_ik"]=index_ik
    linked["support_ki"]=support_ki
    linked["index_ki"]=index_ki
    
    #prepare_ret = linked.drop_duplicates(subset='games', keep='first')[["index_ik","index_ki","support_ik","support_ki"]]
    ret1 = linked.set_index(index_ik)["support_ik"]
    ret2 = linked.set_index(index_ki)["support_ki"]
    ret = ret1.append(ret2)
    ret = ret.groupby(level=[0,1]).sum()

    return ret



In [140]:
def support_map_vectorized_direct_indirect_weighted(linked,direct_thres=1,spread_thres=0,weight_indirect=0.5,verbose=False):
    # columns
    # 'team_j', 'team_i_name', 'team_i_score', 'team_i_H_A_N',
    # 'team_j_i_score', 'team_j_i_H_A_N', 'game_i_j', 'team_k_name',
    # 'team_k_score', 'team_k_H_A_N', 'team_j_k_score', 'team_j_k_H_A_N',
    # 'game_k_j'
    linked["direct"] = linked["team_i_name"] == linked["team_k_name"]
    # | (linked["team_i_name"] == linked["team_j_k_name"]) | (linked["team_k_name"] == linked["team_j_k_name"])
    for_index1 = linked[["team_i_name","team_k_name"]].copy()
    for_index1.loc[linked["direct"]] = linked.loc[linked["direct"],["team_i_name","team_j_name"]]
    for_index1.columns = ["team1","team2"]
    for_index2 = linked[["team_k_name","team_i_name"]].copy()
    for_index2.loc[linked["direct"]] = linked.loc[linked["direct"],["team_j_name","team_i_name"]]
    for_index2.columns = ["team1","team2"]
    index_ik = pd.MultiIndex.from_frame(for_index1,sortorder=0)
    index_ki = pd.MultiIndex.from_frame(for_index2,sortorder=0)
    
    #######################################
    # part to modify
    # direct
    d_ik = linked['team_i_score'] - linked['team_j_i_score']
    support_ik = (linked["direct"] & (d_ik > direct_thres)).astype(int)
    support_ki = (linked["direct"] & (d_ik < -direct_thres)).astype(int)

    # indirect
    d_ij = linked["team_i_score"] - linked["team_j_i_score"]
    d_kj = linked["team_k_score"] - linked["team_j_k_score"]
    
    # always a positive and it captures that if i beat j by 5 points and k beat j by 2 points then this spread is 3
    spread = np.abs(d_ij - d_kj) 
    
    support_ik += weight_indirect*((~linked["direct"]) & (d_ij > 0) & (d_kj < 0) & (spread > spread_thres)).astype(int)
    
    support_ki += weight_indirect*((~linked["direct"]) & (d_kj > 0) & (d_ij < 0) & (spread > spread_thres)).astype(int)
    
    # end part to modify
    #######################################    
    linked["support_ik"]=support_ik
    linked["index_ik"]=index_ik
    linked["support_ki"]=support_ki
    linked["index_ki"]=index_ki
        
    if verbose:
        print('Direct')
        print("Total:",sum(linked["direct"] & (linked["support_ik"]>0)) + sum(linked["direct"] & (linked["support_ki"]>0)),
              "ik:",sum(linked["direct"] & (linked["support_ik"]>0)), 
              "ki:",sum(linked["direct"] & (linked["support_ki"]>0)))
        print('Indirect')
        print("Total:",sum((~linked["direct"]) & (linked["support_ik"]>0)) + sum(~linked["direct"] & (linked["support_ki"]>0)),
              "ik:",sum((~linked["direct"]) & (linked["support_ik"]>0)), 
              "ki:",sum(~linked["direct"] & (linked["support_ki"]>0)))
    
    #indices_ik = linked.index[(linked["support_ik"] > linked["support_ki"]) & ~linked['direct']]    
    #indices_ki = linked.index[(linked["support_ik"] > linked["support_ki"]) & ~linked['direct']] 
    #linked.loc[~linked['direct']] = 0
    #linked.loc[indices_ik] = 0.5*(linked.loc[indices_ik]["support_ik"] - linked.loc[indices_ik]["support_ki"])
    #linked.loc[indices_ik] = 0.5*(linked.loc[indices_ik]["support_ik"] - linked.loc[indices_ik]["support_ki"])
    
    ret1 = linked.set_index(index_ik)["support_ik"]
    ret2 = linked.set_index(index_ki)["support_ki"]
    ret = ret1.append(ret2)
    ret = ret.groupby(level=[0,1]).sum()
    return ret



In [141]:
Vs = {"direct": {},"direct_indirect_weighted": {}}
direct_thresholds = [0,1,2,5]
spread_thresholds = [0,1,2,5]
weight_indirects = [0.25,0.5,1]
for year in games.keys():
    print(year)
    madness_teams = np.unique(list(games[year].team1_name.loc[games[year].team1_madness == 1]) + list(games[year].team2_name.loc[games[year].team2_madness == 1]))
    game_list = list(games[year].index)
    
    game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                            "team1_score":games[year]['points1'],
                            "team1_H_A_N": games[year]['H_A_N1'],
                            "team2_name":games[year]['team2_name'],
                            "team2_score":games[year]['points2'],
                            "team2_H_A_N": games[year]['H_A_N1']})
    Vs["direct"][year] = {}
    Vs["direct_indirect_weighted"][year] = {}
    for direct_thres in direct_thresholds:
        map_func = lambda linked: support_map_vectorized_direct(linked,direct_thres=direct_thres)
        Vs["direct"][year]["direct_thres=%f"%(direct_thres)] = pyrankability.construct.V_count_vectorized(game_df,map_func).loc[madness_teams,madness_teams]
    for direct_thres in direct_thresholds:
        for spread_thres in spread_thresholds:
            for weight_indirect in weight_indirects:
                map_func = lambda linked: support_map_vectorized_direct_indirect_weighted(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
                Vs["direct_indirect_weighted"][year]["direct_thres=%f,spread_thres=%f,weight_indirect=%f"%(direct_thres,spread_thres,weight_indirect)] = pyrankability.construct.V_count_vectorized(game_df,map_func).loc[madness_teams,madness_teams]

2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [128]:
for key in Vs.keys():
    print(key)
    print('Number of elements',len(Vs[key][year].stack()),'Frac of total',len(Vs[key][year].stack())/(len(Vs[key][year])**2-len(Vs[key][year])))

direct
Number of elements 418 Frac of total 0.10367063492063493
direct_indirect_weighted
Number of elements 2910 Frac of total 0.7217261904761905


### Grab the acc teams from 2014 and all games they played

In [72]:
acc_teams = ["Boston_College", 
             "Clemson", 
             "Duke", 
             "Georgia_Tech", 
             "Florida_St", 
             "NC_State", 
             "Syracuse", 
             "Louisville", 
             "Miami_FL", 
             "North_Carolina", 
             "Notre_Dame", 
             "Pittsburgh", 
             "Virginia", 
             "Virginia_Tech", 
             "Wake_Forest"]
#acc_teams = [team.replace("the ","").replace(" ","_").replace("_University","").replace("_Institute","").replace("_of_Technology","_Tech") for team in acc_teams]
acc_teams = ["Duke", "NC_State", "North_Carolina", "Wake_Forest", "Clemson"]
pd.Series(acc_teams)

0              Duke
1          NC_State
2    North_Carolina
3       Wake_Forest
4           Clemson
dtype: object

In [73]:
year="2014"
game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                        "team1_score":games[year]['points1'],
                        "team1_H_A_N": games[year]['H_A_N1'],
                        "team2_name":games[year]['team2_name'],
                        "team2_score":games[year]['points2'],
                        "team2_H_A_N": games[year]['H_A_N1']})

In [74]:
teams = list(game_df.loc[game_df.team1_name.isin(acc_teams) | game_df.team1_name.isin(acc_teams)].team1_name.value_counts().index)
teams += list(game_df.loc[game_df.team1_name.isin(acc_teams) | game_df.team1_name.isin(acc_teams)].team2_name.value_counts().index)
something_missing=False
for acc_team in acc_teams:
    if acc_team not in teams:
        print("missing",acc_team)
        something_missing = True
something_missing

False

In [75]:
acc_game_df = game_df.loc[game_df.team1_name.isin(acc_teams) | game_df.team2_name.isin(acc_teams)]
acc_game_df

,team1_name,team1_score,team1_H_A_N,team2_name,team2_score,team2_H_A_N
53,Duke,74,0,Alabama,64,0
157,NC_State,98,1,Appalachian_St,77,1
402,Clemson,62,-1,Boston_College,60,-1
404,Duke,89,-1,Boston_College,68,-1
413,NC_State,78,1,Boston_College,68,1
...,...,...,...,...,...,...
5284,Pittsburgh,80,1,Wake_Forest,65,1
5286,Syracuse,67,-1,Wake_Forest,57,-1
5287,Tennessee,82,0,Wake_Forest,63,0
5288,Virginia,74,1,Wake_Forest,51,1


In [76]:
acc_game_df.to_csv("acc_game_info.csv",index=False)

In [77]:
import random
seed = random.randint(0,100000)
np.random.seed(88059)
#np.random.seed(seed)
print(seed)
sample_acc_game_df = acc_game_df.sample(frac=0.1)
print(len(acc_game_df))
sample_acc_game_df.to_csv("sample_acc_game_info.csv",index=False)
sample_acc_V = pyrankability.construct.V_count_vectorized(sample_acc_game_df,support_map_vectorized1)
sample_acc_V = sample_acc_V.loc[sample_acc_V.columns.isin(acc_teams),sample_acc_V.columns.isin(acc_teams)]
stacked_sample_acc_V = sample_acc_V.stack()
stacked_sample_acc_V.loc[stacked_sample_acc_V!=0]

5669
138
Direct
13 1
Indirect
2 11


team1  team2   
Duke   NC_State    3.0
dtype: float64

In [78]:
stacked_sample_acc_V.loc[stacked_sample_acc_V!=0].to_csv("stacked_sample_acc_V.csv")

In [79]:
acc_manual_subset_df = pd.read_csv("acc_manual_subset.csv")
acc_manual_subset_df

,team1_name,team1_score,team1_H_A_N,team2_name,team2_score,team2_H_A_N
0,North_Carolina,80,1,Clemson,61,1
1,Wake_Forest,62,1,Clemson,57,1
2,Clemson,72,1,Duke,59,1
3,North_Carolina,74,1,Duke,66,1
4,Wake_Forest,82,1,Duke,72,1
5,Clemson,73,1,NC_State,56,1
6,Duke,95,1,NC_State,60,1
7,North_Carolina,84,1,NC_State,70,1
8,Wake_Forest,70,1,NC_State,69,1
9,Duke,93,1,North_Carolina,81,1


In [80]:
manual_subset_acc_V = pyrankability.construct.V_count_vectorized(acc_manual_subset_df,support_map_vectorized1)
manual_subset_acc_V = manual_subset_acc_V.loc[manual_subset_acc_V.columns.isin(acc_teams),manual_subset_acc_V.columns.isin(acc_teams)]

Direct
34 6
Indirect
20 129


In [81]:
manual_subset_acc_V

team2,Clemson,Duke,NC_State,North_Carolina,Wake_Forest
team1,,,,,
Clemson,NaN,4.0,5.0,3.0,11.0
Duke,5.0,NaN,7.0,11.0,13.0
NC_State,1.0,1.0,NaN,1.0,10.0
North_Carolina,5.0,6.0,9.0,NaN,14.0
Wake_Forest,4.0,5.0,3.0,4.0,NaN


In [82]:
game_df.head().to_csv("sample_game_info.csv",index=False)

In [148]:
thresholds = [0]
results = {"lop":{},"massey":{},"colley":{}}
for t in thresholds:
    results["hillside(t=%d)"%t] = {}
    results["hillside(t=%d,nan=>0)"%t] = {}
for key in results.keys():
    for V_method in Vs.keys():
        results[key][V_method] ={}
        for year in games.keys():
            results[key][V_method][year] = {}

In [149]:
for V_method in Vs.keys():
    for year in games.keys():
        for parameters in Vs[V_method][year].keys():
            print(V_method,year,parameters)
            V = Vs[V_method][year][parameters]
            for t in thresholds:
                C = pyrankability.construct.C_count(V,t)
                C0 = pyrankability.construct.C_count(V.fillna(0),t)
                results["hillside(t=%d)"%(t)][V_method][year][parameters] = pyrankability.rank.solve(V,c_orig=C,method='hillside',lazy=False, cont=True)
                results["hillside(t=%d,nan=>0)"%(t)][V_method][year][parameters] = pyrankability.rank.solve(V.fillna(0),c_orig=C0,method='hillside',lazy=False, cont=True)
            results["lop"][V_method][year][parameters] = pyrankability.rank.solve(V.fillna(0),method='lop',lazy=False, cont=True)
            try:
                results["massey"][V_method][year][parameters] = MasseyRankingAlgorithm().rank(V.fillna(0).values),MasseyRatingAlgorithm().rank(V.values)
            except Exception as e:
                results["massey"][V_method][year][parameters] = str(e)  
            try:
                results["colley"][V_method][year][parameters] = ColleyRankingAlgorithm().rank(V.fillna(0).values),ColleyRatingAlgorithm().rank(V.values)
            except Exception as e:
                results["colley"][V_method][year][parameters] = str(e)                
            

direct 2002 direct_thres=0.000000
direct 2002 direct_thres=1.000000
direct 2002 direct_thres=2.000000
direct 2002 direct_thres=5.000000
direct 2003 direct_thres=0.000000
direct 2003 direct_thres=1.000000
direct 2003 direct_thres=2.000000
direct 2003 direct_thres=5.000000
direct 2004 direct_thres=0.000000
direct 2004 direct_thres=1.000000
direct 2004 direct_thres=2.000000
direct 2004 direct_thres=5.000000
direct 2005 direct_thres=0.000000
direct 2005 direct_thres=1.000000
direct 2005 direct_thres=2.000000
direct 2005 direct_thres=5.000000
direct 2006 direct_thres=0.000000
direct 2006 direct_thres=1.000000
direct 2006 direct_thres=2.000000
direct 2006 direct_thres=5.000000
direct 2007 direct_thres=0.000000
direct 2007 direct_thres=1.000000
direct 2007 direct_thres=2.000000
direct 2007 direct_thres=5.000000
direct 2008 direct_thres=0.000000
direct 2008 direct_thres=1.000000
direct 2008 direct_thres=2.000000
direct 2008 direct_thres=5.000000
direct 2009 direct_thres=0.000000
direct 2009 di

### Save the results for later

In [151]:
list(results.keys())

['lop', 'massey', 'colley', 'hillside(t=0)', 'hillside(t=0,nan=>0)']

In [152]:
import joblib
joblib.dump({"results":results,"Vs":Vs,"games":games},"MarchMadnessAnalysis.joblib.z")

['MarchMadnessAnalysis.joblib.z']